In [3]:
%matplotlib inline
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import sys, os

from lammpsrun import LAMMPS, Prism

from ase import Atoms, units
from ase.visualize import view
from ase.io.trajectory import Trajectory
from ase.io import write, read
from ase.neighborlist import neighbor_list
from ase.build import surface
from ase.spacegroup import crystal
from ase.geometry import *

from pymatgen.core.surface import *
from pymatgen.io.ase import AseAtomsAdaptor as AAA

homepath = "D:\\Cloud Storage\\GitRepositories\\LAMMPS_Simulation\\"
hpc_path = "G:\\home\\LAMMPS_Simulation\\HPC_Jupyter\\"
print(homepath + '\n' + hpc_path)

D:\Cloud Storage\GitRepositories\LAMMPS_Simulation\
G:\home\LAMMPS_Simulation\HPC_Jupyter\


Crystallographic data is obtained from https://materials.springer.com/isp/crystallographic/docs/sd_1628167

# Bulk siderite crystal

This part is done directly within CX1 notebook.

# Surfaces/Slabs

pymatgen referenc: https://matgenb.materialsvirtuallab.org/2017/04/03/Slab-generation-and-Wulff-shape.html

In [4]:
def find_top_site(sites_list, atomstring):
    c_indices = []
    c_coord_z = []

    for i, site in enumerate(sites_list):
        if site.species_string == atomstring:
            c_indices.append(i)
            c_coord_z.append(site.z)

        # Ensuring sequence is sorted
        c_indices_z = sorted(zip(c_coord_z, c_indices))
        
    index_to_remove = c_indices_z[-1][1]
    
    return index_to_remove

Past work done with surface 104, 110, 100, 101, 001 and 012. High surface energy on 100 and 101

From bulk optimization we have new cell parameters as ratio of original:
- Lx = 0.994
- Ly = 0.994
- Lz = 1.035

In [8]:
# From optimization:
opt_adjustment = [0.97604897, 0.97605518, 1.07602404]
opt_adjustment = [(opt_adjustment[0]+opt_adjustment[1])/2, (opt_adjustment[0]+opt_adjustment[1])/2, opt_adjustment[2]]
opt_adjustment

[0.976052075, 0.976052075, 1.07602404]

In [9]:
siderite_lattice = Lattice.from_lengths_and_angles([4.676*0.976052075, 4.676*0.976052075, 15.31*1.07602404],
                                                   [90, 90, 120])
siderite = Structure.from_spacegroup(sg=167,
                                     lattice=siderite_lattice,
                                     species=["Fe", "C", "O"],
                                     coords=[[0, 0, 0], 
                                             [0, 0, 0.25],
                                             [0.2741, 0, 0.25]])
siderite.add_oxidation_state_by_element({"Fe": 2, "C": 4, "O": -2})

In [10]:
siderite_surfaces_miller_indices = ['104', '110', '100', '101', '001', '012']
siderite_surface = dict.fromkeys(siderite_surfaces_miller_indices)

In [11]:
for key in siderite_surface.keys():
    miller_index = [int(x) for x in key]
    
    slabgen = SlabGenerator(siderite, miller_index,
                            min_slab_size=21,
                            min_vacuum_size=10,
                            center_slab=True,
                            in_unit_planes=True)

    slab_list = slabgen.get_slabs(bonds={("C", "O"): 1.5},
                                  symmetrize=True)
    
    siderite_surface[key] = [x for x in slab_list if abs(x.charge) <= 4]

    print("\nSurface:", key, "\tNumber of allowed terminations:", len(siderite_surface[key]))
    
    for slab in siderite_surface[key]:
        print(slab.formula)


Surface: 104 	Number of allowed terminations: 1
Fe48 C48 O144

Surface: 110 	Number of allowed terminations: 3
Fe40 C40 O122
Fe42 C42 O126
Fe42 C42 O124

Surface: 100 	Number of allowed terminations: 1
Fe132 C132 O396

Surface: 101 	Number of allowed terminations: 2
Fe43 C44 O130
Fe43 C42 O128

Surface: 001 	Number of allowed terminations: 2
Fe125 C126 O378
Fe125 C124 O372

Surface: 012 	Number of allowed terminations: 4
Fe40 C42 O126
Fe40 C42 O122
Fe42 C40 O124
Fe42 C40 O120


In [12]:
for key in siderite_surface.keys():
    print("*******************************************")
    print("Surface:", key)
    print("*******************************************")
    for slab in siderite_surface[key]:
        slab_ase = AAA.get_atoms(slab)
        layer_info, distances = get_layers(slab_ase, (0,0,1), tolerance=0.1)
        layer_atoms = list(sorted(zip(layer_info, slab_ase.get_chemical_symbols())))
        layer_atoms_dict = {}
        for x, y in layer_atoms:
            layer_atoms_dict.setdefault(x, []).append(y)

        for layer in sorted(layer_atoms_dict.keys()):
            max_layers = len(layer_atoms_dict.keys())
            if layer < 6:
                print('', layer+1, layer_atoms_dict[layer])
            elif max_layers - layer == 6:
                print(".")
                print(".")
                print(layer - max_layers, layer_atoms_dict[layer])
            elif max_layers - layer < 6:
                print(layer - max_layers, layer_atoms_dict[layer])
                
        slab_composition = slab.composition.to_reduced_dict
        if slab_composition['Fe'] == slab_composition['C'] and slab_composition['Fe'] * 3 == slab_composition['O']:
            print("Stoichiometric: True")
        else:
            print("Stoichiometric: False")
        print("Polar:", slab.is_polar())
        print("Charge:", slab.charge)
        print("\n")

*******************************************
Surface: 104
*******************************************
 1 ['O', 'O']
 2 ['C', 'C', 'Fe', 'Fe', 'O', 'O']
 3 ['O', 'O']
 4 ['O', 'O']
 5 ['C', 'C', 'Fe', 'Fe', 'O', 'O']
 6 ['O', 'O']
.
.
-6 ['O', 'O']
-5 ['C', 'C', 'Fe', 'Fe', 'O', 'O']
-4 ['O', 'O']
-3 ['O', 'O']
-2 ['C', 'C', 'Fe', 'Fe', 'O', 'O']
-1 ['O', 'O']
Stoichiometric: True
Polar: False
Charge: 0


*******************************************
Surface: 110
*******************************************
 1 ['O']
 2 ['O']
 3 ['O', 'O']
 4 ['C', 'C', 'Fe', 'Fe']
 5 ['O', 'O']
 6 ['O']
.
.
-6 ['O']
-5 ['O', 'O']
-4 ['C', 'C', 'Fe', 'Fe']
-3 ['O', 'O']
-2 ['O']
-1 ['O']
Stoichiometric: False
Polar: False
Charge: -4


 1 ['O']
 2 ['O', 'O']
 3 ['C', 'C', 'Fe', 'Fe']
 4 ['O', 'O']
 5 ['O']
 6 ['O']
.
.
-6 ['O']
-5 ['O']
-4 ['O', 'O']
-3 ['C', 'C', 'Fe', 'Fe']
-2 ['O', 'O']
-1 ['O']
Stoichiometric: True
Polar: False
Charge: 0


 1 ['O', 'O']
 2 ['C', 'C', 'Fe', 'Fe']
 3 ['O', 'O']
 4 ['O']
 5 

In [13]:
surface_110_NS = siderite_surface['110'][2].copy()
surface_110_NS.symmetrically_remove_atoms([find_top_site(surface_110_NS.sites, 'C4+')])

In [14]:
surface_012_NS = siderite_surface['012'][2].copy()
surface_012_NS.symmetrically_remove_atoms([find_top_site(surface_012_NS.sites, 'O2-')])

In [15]:
surface_012_S = siderite_surface['012'][0].copy()
surface_012_S_top_C = find_top_site(surface_012_S.sites, 'C4+')
surface_012_S_top_O3 = surface_012_S.get_neighbors(surface_012_S.sites[surface_012_S_top_C],1.5, include_index=True)
surface_012_S_top_CO3 = [surface_012_S_top_C] + [x[2] for x in surface_012_S_top_O3]
surface_012_S.symmetrically_remove_atoms(surface_012_S_top_CO3)

In [16]:
# 104 YES
# 110[1] YES NS(110) to be reconstructed from 110[2]
# 100 keep
# 101 keep
# 001[0] YES S(011) to be further constructed
# NS(012) to be reconstucted from 012[2]
# S(012) to be reconstructed from 012[0]

siderite_surface_kept = {}
siderite_surface_kept['104_S'] = siderite_surface['104'][0].copy()
siderite_surface_kept['110_S'] = siderite_surface['110'][1].copy()
siderite_surface_kept['110_NS'] = surface_110_NS
siderite_surface_kept['110_NS'] = siderite_surface['110'][0].copy()
siderite_surface_kept['101_NS0'] = siderite_surface['101'][0].copy()
siderite_surface_kept['101_NS1'] = siderite_surface['101'][1].copy()
siderite_surface_kept['001_NS'] = siderite_surface['001'][0].copy()
siderite_surface_kept['012_S'] = surface_012_S
siderite_surface_kept['012_NS'] = surface_012_NS
siderite_surface_kept['100_S'] = siderite_surface['100'][0].copy()

In [17]:
list_of_layers = list(range(10, 30))

In [19]:
from concurrent.futures import ThreadPoolExecutor as PoolExecutor

# create a thread pool of 4 threads
with PoolExecutor(max_workers=4) as executor:
    for x, result in zip(list_of_layers, executor.map(gen_surfaces, list_of_layers)):
        pass

Layers:Layers:Layers:  Layers: 10 1113
12


Layers: 14
Layers: 15
Layers: 16
Layers: 17
Layers: 18
Layers: 19
Layers: 20
Layers: 21
Layers: 22
Layers: 23
Layers: 24
Layers: 25
Layers: 26
Layers: 27
Layers: 28
Layers: 29


In [18]:
def gen_surfaces(layers):
    print("Layers:", layers)
    for key in siderite_surface.keys():
        miller_index = [int(x) for x in key]

        slabgen = SlabGenerator(siderite, miller_index,
                                min_slab_size=layers,
                                min_vacuum_size=20,
                                center_slab=True,
                                in_unit_planes=True)

        slab_list = slabgen.get_slabs(bonds={("C", "O"): 1.5},
                                      symmetrize=True)

        siderite_surface[key] = [x for x in slab_list if abs(x.charge) <= 4]

    surface_110_NS = siderite_surface['110'][2].copy()
    surface_110_NS.symmetrically_remove_atoms([find_top_site(surface_110_NS.sites, 'C4+')])
    surface_012_NS = siderite_surface['012'][2].copy()
    surface_012_NS.symmetrically_remove_atoms([find_top_site(surface_012_NS.sites, 'O2-')])
    surface_012_S = siderite_surface['012'][0].copy()
    surface_012_S_top_C = find_top_site(surface_012_S.sites, 'C4+')
    surface_012_S_top_O3 = surface_012_S.get_neighbors(surface_012_S.sites[surface_012_S_top_C],1.5, include_index=True)
    surface_012_S_top_CO3 = [surface_012_S_top_C] + [x[2] for x in surface_012_S_top_O3]
    surface_012_S.symmetrically_remove_atoms(surface_012_S_top_CO3)

    siderite_surface_kept = {}
    siderite_surface_kept['104_S'] = siderite_surface['104'][0].copy()
    siderite_surface_kept['110_S'] = siderite_surface['110'][1].copy()
    siderite_surface_kept['110_NS'] = surface_110_NS
    siderite_surface_kept['110_NS'] = siderite_surface['110'][0].copy()
    siderite_surface_kept['101_NS0'] = siderite_surface['101'][0].copy()
    siderite_surface_kept['101_NS1'] = siderite_surface['101'][1].copy()
    siderite_surface_kept['001_NS'] = siderite_surface['001'][0].copy()
    siderite_surface_kept['012_S'] = surface_012_S
    siderite_surface_kept['012_NS'] = surface_012_NS
    siderite_surface_kept['100_S'] = siderite_surface['100'][0].copy()

    directory_path = os.path.join(hpc_path, "siderite", "surfaces")
    for surface, slab in siderite_surface_kept.items():
        slab_ase = AAA.get_atoms(slab)
        layer_info, layer_distances = get_layers(slab_ase, (0,0,1), tolerance=0.1)
        nlayers = max(layer_info)

#         print("Surface:", surface, "\tLayers:", nlayers)
        file_path = os.path.join(directory_path, "surface{0}_L{1}.extxyz".format(surface, nlayers))
        slab_ase.write(file_path, format='extxyz')

## Viewing the Results

In [2]:
ase_traj = Trajectory(r"G:\ephemeral\2371379.cx1\siderite_bulk_331.traj", 'r')